In [6]:
import csv
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from collections import Counter
import math
from math import exp
from scipy.spatial.distance import cdist
import scipy
from copy import deepcopy

In [7]:
from sklearn.datasets import load_iris

iris = load_iris()

from itertools import combinations


In [8]:
data = iris.data #iris is a dictionary and hence we re gettiing data by keys.
labels = iris.target

In [9]:
data.shape, labels.shape

((150, 4), (150,))

In [10]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [11]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [12]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [112]:
class KM_plusplus():
    '''This class uses KMeans++ algorithm for initial centroid location which are not random. This imporves the cluster
    quality and gives persistent clustering.
    
    Parameters
    ----------------
    n_clusters: int
        number of clusters to find. They are group of datapoints that will form clusters based of the distance from 
        centroid.
    
    Attributes
    ----------------
    centroids_: array, shape (n_clusters, xtrain.shape[1])
        centroid, center point of cluster from where distance of cluster points is minimum. 
        
    classes_: dict, should have same number of datapoints as the original dataset.
        group of datapoints forming the clusters.
    '''
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.centroids_ = 0 
        self.classes_ = 0
        
    def fit(self, xtrain, method='km++'):
        '''fit method implements Lloyd's algorithm, whenever the centroids (center of mass of cluster) stop moving, 
        stop iteration. Euclidean pairwise distance between each datapoint and every centroid is measured and datapoint
        is allocated to nearest centroid
        
        Parameters
        ----------------
        xtrain: pd.dataframe
            
        method: str (default 'km++') {'km++', 'random'}
            specify which algorithm to use for calculating initial centroids. 
        '''
        
        if method=='km++':
            self.centroids_ = self.centroid_initialize(xtrain, self.n_clusters)
        elif method=='random':
            self.centroids_= xtrain.sample(self.n_clusters).values #random initialization of clusters
        else:
            print('This method is not implemented')
            
        previous = np.zeros(self.centroids_.shape)
        error = np.linalg.norm(self.centroids_ - previous) #initialize error for iteration
        #print(error)
        count = 0
        while error >= 0.005:
        #implementing Lloyd algorithms(stop when new cluster does not move)
            count += 1
            self.classes_ = {}
            for index in range(self.n_clusters):
                self.classes_[index] = []
            for row in xtrain.values:
                    #distances = [np.linalg.norm(row - centroid) for centroid in centroids.values]
                    distances = [np.linalg.norm(row - centroid) for centroid in self.centroids_]
                    classify = distances.index(min(distances))
                    self.classes_[classify].append(row)
            previous = deepcopy(self.centroids_)
            for classify in self.classes_:
                #print(classify, classes[classify])
                #print(np.mean(classes[classify], axis =0))
                self.centroids_[classify] = np.mean(self.classes_[classify], axis=0)
            #print(previous)
            #print(centroids)

            error = np.linalg.norm(previous-self.centroids_)
            print(error)
        #print(self.classes_)
        print(count) #printing number of iteration for reaching minimum error
        return self.centroids_ 
    
    def centroid_initialize(self, xtrain, random_seed=None):
        '''k-means++ algorithm, searching for farthest point (.85 of farthest point, as farthest point can be outliers) 
        for next centroid, vectorized'''
    
        centroid = xtrain.sample(1).values # add a random seed    
        centroids = {}
        for i in range(1,self.n_clusters):
            for index, item in enumerate(centroid):
                item = item.reshape(-1, item.shape[0])
                euclidean_dist = cdist(xtrain.values, item)
                centroids[index] = euclidean_dist
            minim = []
            for i in zip(*centroids.values()):
                minim.append(min(i))
            r = 0.85*float(max(minim))

            index = np.where(np.array(minim) >= r)[0][0]
            centroid = np.vstack((centroid, xtrain.iloc[index].values))
        return centroid
    

In [113]:
km  = KM_plusplus(3)
km.fit(iris_df)

0.738659977595
0.370141728842
0.112262087317
0.0874869355307
0.0
5


array([[ 5.9016129 ,  2.7483871 ,  4.39354839,  1.43387097],
       [ 5.006     ,  3.418     ,  1.464     ,  0.244     ],
       [ 6.85      ,  3.07368421,  5.74210526,  2.07105263]])

In [116]:
km.centroids_

array([[ 5.9016129 ,  2.7483871 ,  4.39354839,  1.43387097],
       [ 5.006     ,  3.418     ,  1.464     ,  0.244     ],
       [ 6.85      ,  3.07368421,  5.74210526,  2.07105263]])

In [115]:
km.classes_

{0: [array([ 7. ,  3.2,  4.7,  1.4]),
  array([ 6.4,  3.2,  4.5,  1.5]),
  array([ 5.5,  2.3,  4. ,  1.3]),
  array([ 6.5,  2.8,  4.6,  1.5]),
  array([ 5.7,  2.8,  4.5,  1.3]),
  array([ 6.3,  3.3,  4.7,  1.6]),
  array([ 4.9,  2.4,  3.3,  1. ]),
  array([ 6.6,  2.9,  4.6,  1.3]),
  array([ 5.2,  2.7,  3.9,  1.4]),
  array([ 5. ,  2. ,  3.5,  1. ]),
  array([ 5.9,  3. ,  4.2,  1.5]),
  array([ 6. ,  2.2,  4. ,  1. ]),
  array([ 6.1,  2.9,  4.7,  1.4]),
  array([ 5.6,  2.9,  3.6,  1.3]),
  array([ 6.7,  3.1,  4.4,  1.4]),
  array([ 5.6,  3. ,  4.5,  1.5]),
  array([ 5.8,  2.7,  4.1,  1. ]),
  array([ 6.2,  2.2,  4.5,  1.5]),
  array([ 5.6,  2.5,  3.9,  1.1]),
  array([ 5.9,  3.2,  4.8,  1.8]),
  array([ 6.1,  2.8,  4. ,  1.3]),
  array([ 6.3,  2.5,  4.9,  1.5]),
  array([ 6.1,  2.8,  4.7,  1.2]),
  array([ 6.4,  2.9,  4.3,  1.3]),
  array([ 6.6,  3. ,  4.4,  1.4]),
  array([ 6.8,  2.8,  4.8,  1.4]),
  array([ 6. ,  2.9,  4.5,  1.5]),
  array([ 5.7,  2.6,  3.5,  1. ]),
  array([ 5.5,  2

#### I wrote multiple vectorized implementation for finding initial centroids. Using scipy.cdist, np.inner and pairwise.euclidean_distance methods. 


In [119]:
def centroid_initialize(xtrain, n_centroid):
## k-means++ approach, searching for farthest point for next centroid with probability function, tweeked to avoid getting outliers
    centroid = xtrain.sample(1).as_matrix()
    
    for i in range(1,n_centroid):
        #print(centroid)
        dist = np.array([min([np.inner(c-x,c-x)for c in centroid]) for x in xtrain.values])
        #print(dist)
        dist_prob = dist/dist.sum()
        cum_dist = dist_prob.cumsum()
        index = np.where(cum_dist >= 0.85)[0][0]
        #print(index)
        centroid = np.vstack((centroid, xtrain.iloc[index].values))
    return centroid

In [120]:
def centroid_initialize2(xtrain, n_centroid, random_seed=None):
## k-means++ approach, searching for farthest point for next centroid with probability function, vectorized
    centroid = xtrain.sample(1).values # .values add a random seed    
    centroids = {}
    for i in range(1,n_centroid):
        for index, item in enumerate(centroid):
            item = item.reshape(-1, item.shape[0])
            euclidean_dist = cdist(xtrain.values, item)
            centroids[index] = euclidean_dist
        minim = []
        for i in zip(*centroids.values()):
            minim.append(min(i))
        r = 0.85*float(max(minim))

        index = np.where(np.array(minim) >= r)[0][0]
        centroid = np.vstack((centroid, xtrain.iloc[index].values))
    return centroid

In [121]:
from sklearn.metrics.pairwise import euclidean_distances

def centroid_initialize3(xtrain, n_centroid, random_seed=None):
    centroids = xtrain.sample(1).values.reshape( (1,) + xtrain.shape[1:])
    
    for i in range(n_centroid-1): 
        d_x = np.min(euclidean_distances(xtrain, centroids), axis=1)
        d_x = d_x / np.sum(d_x)
        new_centroid = xtrain.iloc[np.random.choice(len(xtrain), size=1, p=d_x)]
        centroids = np.vstack((centroids, new_centroid))
    return centroids

In [122]:
def centroid_initialize1(xtrain, n_centroid):
## k-means++ approach, searching for farthest point for next centroid with probability function
    centroid = xtrain.sample(1)
    print(centroid.values)
    
    centroids = {}
    #for i in range(1,n_centroid):
    for index, item in enumerate(centroid.values):
        item = item.reshape(-1, item.shape[0])
            #print(np.linalg.norm(xtrain.values[1] - item)
        euclidean_dist = cdist(xtrain.values, item)
        centroids[index] = euclidean_dist

        #print(centroids)
    minim = []
        #print(list(zip(*centroids.values())))
    for i in zip(*centroids.values()):
        minim.append(min(i))
    r = 0.85*float(max(minim))

    index = np.where(np.array(minim) >= r)[0][0]
    centroid = np.vstack((centroid, xtrain.iloc[index].values))
    print(r,(minim), index)
    return centroid

In [123]:
centroid_initialize(iris_df, 2)

array([[ 6.7,  3.3,  5.7,  2.1],
       [ 5.2,  2.7,  3.9,  1.4]])

## Sklearn KMeans clustering comparison

In [125]:
km = KMeans(n_clusters=3)
km.fit(iris_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [126]:
km.cluster_centers_

array([[ 5.9016129 ,  2.7483871 ,  4.39354839,  1.43387097],
       [ 5.006     ,  3.418     ,  1.464     ,  0.244     ],
       [ 6.85      ,  3.07368421,  5.74210526,  2.07105263]])

In [127]:
km.predict

<bound method KMeans.predict of KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)>

In [1]:
import pandas as pd